In [5]:
import smtplib
import shutil
import cv2
import torch
import os
import time
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
from google.colab.patches import cv2_imshow
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

def send_email(sender_email, receiver_email, email_password, frame_filenames, video_link):
    """Send an email with a link to CCTV footage and attached images of detected frames."""
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = "Gun Detection Alert"

    # HTML body with clickable video link
    body = f"""
    <html>
        <body>
            <p>Gun detected in the surveillance footage. Please check the images attached and click the link below for the CCTV footage.</p>
            <p><a href="{video_link}" target="_blank">View CCTV Footage</a></p>
        </body>
    </html>
    """
    msg.attach(MIMEText(body, 'html'))

    # Attach each image file to the email
    for filename in frame_filenames:
        try:
            with open(filename, 'rb') as img:
                mime_image = MIMEImage(img.read())
                mime_image.add_header('Content-Disposition', f'attachment; filename={os.path.basename(filename)}')
                msg.attach(mime_image)
        except FileNotFoundError:
            print(f"Error: File {filename} not found, skipping attachment.")

    # Send the email
    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()  # Secure the connection
            server.login(sender_email, email_password)
            server.send_message(msg)
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

In [7]:
# Object detection class
class ObjectDetection:
    def __init__(self, video_path, sender_email, receiver_email, email_password, live_video_link):
        """Initializes an ObjectDetection instance with a given video file path and email credentials."""
        self.video_path = video_path
        self.email_sent = False

        # Email credentials
        self.sender_email = sender_email
        self.receiver_email = receiver_email
        self.email_password = email_password

        # Model information
        self.model = YOLO("/content/best_manas_91.pt")

        # Visual information
        self.annotator = None
        self.start_time = 0
        self.end_time = 0

        # Device information
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        # Frame count and time tracking
        self.gun_frame_count = 0
        self.interval_start_time = time.time()  # Start timing for the 10-second interval

    def predict(self, im0):
        """Run prediction using a YOLO model for the input image im0."""
        results = self.model(im0)
        return results

    def plot_bboxes(self, results, im0):
        """Plots bounding boxes on an image given detection results; returns annotated image, class IDs, and confidence scores."""
        class_ids = []
        confidences = []  # List to store confidence scores
        self.annotator = Annotator(im0, 3, results[0].names)

        boxes = results[0].boxes.xyxy.cpu()  # Bounding box coordinates
        clss = results[0].boxes.cls.cpu().tolist()  # Class labels (as indices)
        confs = results[0].boxes.conf.cpu().tolist()  # Confidence scores
        names = results[0].names  # Class names

        for box, cls, conf in zip(boxes, clss, confs):
            class_ids.append(cls)
            confidences.append(f"{conf:.2f}")  # Append confidence score formatted to 2 decimal places
            label = f"{names[int(cls)]} {conf:.2f}"  # Include confidence in the label
            self.annotator.box_label(box, label=label, color=colors(int(cls), True))

        return im0, class_ids, confidences

    def __call__(self):
        """Run object detection on video frames from a video file, plotting and showing the results."""
        cap = cv2.VideoCapture(self.video_path)  # Load video from the file
        assert cap.isOpened()
        output_dir = '/content/safe'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        count = 0  # Initialize frame count
        detected_frames = []  # List to store saved frame file paths

        while True:
            # if self.email_sent:
            #     break

            self.start_time = time.time()
            ret, im0 = cap.read()
            if not ret:  # If no more frames are available, break the loop
                break

            results = self.predict(im0)
            im0, class_ids, confidences = self.plot_bboxes(results, im0)

            # Check if gun class (class '0') is detected
            if 0 in class_ids or 1 in class_ids:
                for cls, conf in zip(class_ids, confidences):
                    if (cls == 0 or cls == 1) and float(conf) > 0.5:  # Check if it's a gun class and confidence > 50%
                        self.gun_frame_count += 1  # Count frames with gun detection
                        frame_filename = os.path.join(output_dir, f"frame_{count:04d}.jpg")
                        cv2.imwrite(frame_filename, im0)  # Save the frame with gun detection
                        detected_frames.append(frame_filename)
                        break  # Break after first match since frame is already saved

            if self.gun_frame_count >= 10:
                send_email(self.sender_email, self.receiver_email, self.email_password, detected_frames[-10:], live_video_link)  # Send last 10 detected frames
                self.email_sent = True

            # Check if 10 seconds have passed
            if time.time() - self.interval_start_time >= 10:
                # If gun detected in at least 5 frames within 10 seconds, send an email
                # Clear the folder after sending the email
                shutil.rmtree(output_dir)  # Remove the folder and its contents
                os.makedirs(output_dir)  # Recreate an empty folder
                detected_frames.clear()  # Clear the list of detected frames

                # Reset the interval and frame count after each 10 seconds
                self.interval_start_time = time.time()
                self.gun_frame_count = 0  # Reset for the next 10-second interval

            count += 1
            if cv2.waitKey(5) & 0xFF == 27:  # Press 'ESC' to exit
                break

        cap.release()
        cv2.destroyAllWindows()

# Example usage of the class
# Replace with actual values for sender, receiver, and email password
sender_email = "manas.divekar76@gmail.com"  # Replace with your email
receiver_email = "me.atharvajadhav@gmail.com"  # Replace with receiver's email
email_password = "jemu imks maqm kaow"  # Replace with your email password
video_path = '/content/armed_robbery_raw1.mp4'  # Path to your video
live_video_link = "http://127.0.0.1:5000/"  # Replace with your actual video link

# Initialize object detection
object_detection = ObjectDetection(video_path, sender_email, receiver_email, email_password, live_video_link)
# Run the detection
object_detection()



0: 384x640 (no detections), 355.3ms
Speed: 2.9ms preprocess, 355.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 gun, 333.7ms
Speed: 3.9ms preprocess, 333.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 gun, 337.1ms
Speed: 3.0ms preprocess, 337.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 gun, 368.2ms
Speed: 3.3ms preprocess, 368.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 339.5ms
Speed: 3.6ms preprocess, 339.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 guns, 348.0ms
Speed: 2.8ms preprocess, 348.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 gun, 367.6ms
Speed: 3.4ms preprocess, 367.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 gun, 342.5ms
Speed: 3.7ms preprocess, 342.5ms inference, 1.0ms postprocess per image at sh

KeyboardInterrupt: 